In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [269]:
#a = pd.read_csv('/content/drive/MyDrive/11.Phd/Movie-Book/movie.csv')

In [270]:
#a.head(1)

In [3]:
import pandas as pd

In [4]:
book = pd.read_csv('/content/drive/MyDrive/11.Phd/Movie-Book/book.csv')
movie = pd.read_csv('/content/drive/MyDrive/11.Phd/Movie-Book/movie.csv')
book = book.drop('Unnamed: 0', axis=1)
movie = movie.drop('Unnamed: 0', axis=1)
#book = book.drop('Unnamed: 0.1', axis=1)
#movie = movie.drop('Unnamed: 0.1', axis=1)

<ipython-input-4-f5613deab5c1>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/drive/MyDrive/11.Phd/Movie-Book/book.csv')


In [9]:
movie.head(1)

,userId,itemId,rating,user_embedding,item_embedding
0,A30TK6U7DNS82R,767803434,4.0,"[0.0041, 0.0, 0.1111, 0.0]","[0.819, 0.045, 0.8205, 0.5205]"


In [6]:
movie = movie[['userId','movieId','rating_movie','user_embedding','movie_item_embedding']]
#book = book[['userId','itemId','rating','user_embedding','book_embedding']]

In [10]:
book.head(1)

,userId,itemId,rating,item_embedding,user_embedding
0,A30TK6U7DNS82R,0826414346,5.0,"[1.0, 0.4375, 0.4373, 0.0011]","[0.0041, 0.0, 0.1111, 0.0]"


In [8]:
book.rename(columns={'book_embedding': 'item_embedding'}, inplace=True)
movie.rename(columns={'movieId':'itemId', 'rating_movie': 'rating','movie_item_embedding': 'item_embedding'}, inplace=True)

In [277]:
#print(book['item_embedding'])

In [11]:
movie.shape, book.shape

((771358, 5), (785069, 5))

In [279]:
book = book.drop_duplicates()
movie = movie.drop_duplicates()

In [12]:
movie.shape, book.shape

((771358, 5), (785069, 5))

In [281]:
book = book.fillna(0)
movie = movie.fillna(0)

In [13]:
book['userId'].nunique()


841

In [283]:
movie['userId'].nunique()

1588

In [14]:
book['userId'] = pd.factorize(book['userId'])[0]
movie['userId'] = pd.factorize(movie['userId'])[0]
book['itemId'] = pd.factorize(book['itemId'])[0]
movie['itemId'] = pd.factorize(movie['itemId'])[0]

In [15]:
movie["rating"] = pd.to_numeric(movie["rating"], errors='coerce')
book["rating"] = pd.to_numeric(book["rating"], errors='coerce')

In [286]:
#book = book[book['userId'].isin(movie['userId'])]
#movie = movie[movie['userId'].isin(book['userId'])]

In [287]:
#book = book.sample(frac=0.1, random_state=42)  # Use a specific random_state for reproducibility

In [288]:
#movie = movie.sample(frac=0.1, random_state=42)

In [289]:
a = .9 * 2910054

In [290]:
a

2619048.6

In [291]:
#movie = movie.head(873059)
#book = book.head(2910054)

In [292]:
book.shape

(500574, 5)

utils

In [16]:
"""
    Some handy functions for pytroch model training ...
"""
import torch


# Checkpoints
def save_checkpoint(model, model_dir):
    torch.save(model.state_dict(), model_dir)


def resume_checkpoint(model, model_dir, device_id):
    state_dict = torch.load(model_dir,
                            map_location=lambda storage, loc: storage.cuda(device=device_id))  # ensure all storage are on gpu
    model.load_state_dict(state_dict)


# Hyper params
def use_cuda(enabled, device_id=0):
    if enabled:
        assert torch.cuda.is_available(), 'CUDA is not available'
        torch.cuda.set_device(device_id)


def use_optimizer(network, params):
    if params['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(network.parameters(), lr=params['lr'], weight_decay=params['l2_regularization'])
    elif params['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(network.parameters(), lr=params['lr'])
    elif params['optimizer'] == 'rmsprop':
        optimizer = torch.optim.RMSprop(network.parameters(),
                                        lr=params['lr'],
                                        alpha=params['alpha'],
                                        momentum=params['momentum'])
    return optimizer

MLP

In [17]:

class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()
        self.config = config
        self.latent_dim = config['latent_dim']
        self.nlayers = config['nlayers']
        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
        self.affine_output = torch.nn.Linear(in_features=config['layers'][-1], out_features=1)
        self.logistic = torch.nn.Sigmoid()
        self.bridge = torch.nn.Linear(config['latent_dim'], config['latent_dim'])
        torch.nn.init.orthogonal_(self.bridge.weight)

    def init_weight(self):
        pass

    def forward(self, user_embeddings, item_embeddings, dual=False):
        if dual:
            user_embeddings = self.bridge(user_embeddings)
        vector = torch.cat([user_embeddings, item_embeddings], dim=-1)  # the concat latent vector
        vector = vector.float()
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = torch.nn.Dropout(p=0.1)(vector)
            vector = torch.nn.ReLU()(vector)
            #vector = torch.nn.BatchNorm1d()(vector)
        rating = self.affine_output(vector)
        #rating = self.logistic(rating)
        return rating

engine

In [18]:
from torch.autograd import Variable
#from mlp import MLP
#from utils import save_checkpoint, use_optimizer, use_cuda
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import numpy as np
Book_MSE_list=[]
Book_MAE_list=[]
#Book_Precision_list=[]
#Book_Recall_list=[]
Music_MSE_list=[]
Music_MAE_list=[]
#Music_Precision_list=[]
#Music_Recall_list=[]

In [296]:
#from sklearn.metrics import ndcg_score

In [19]:
class Engine(object):
    """Meta Engine for training & evaluating NCF model

    Note: Subclass should implement self.model !
    """

    def __init__(self, config):
        self.config = config  # model configuration
        self.modelA = MLP(config)
        self.modelB = MLP(config)
        if config['use_cuda'] is True:
            use_cuda(True, config['device_id'])
            self.modelA.cuda()
            self.modelB.cuda()
        print(self.modelA)
        if config['pretrain']:
            self.model.load_pretrain_weights()
        self.optA = use_optimizer(self.modelA, config)
        self.optB = use_optimizer(self.modelB, config)
        self.crit = torch.nn.MSELoss()
        self.alpha = config['alpha']

    def train_single_batch(self, book_user_embeddings, book_item_embeddings, book_rating,
                          movie_user_embeddings, movie_item_embeddings, movie_rating):
        self.optA.zero_grad()
        self.optB.zero_grad()
        book_ratings_pred1 = self.modelA(book_user_embeddings, book_item_embeddings)
        lossA1 = self.crit(book_ratings_pred1.squeeze(1), book_rating)
        book_ratings_pred2 = self.modelB(book_user_embeddings, book_item_embeddings, dual=True)
        lossA2 = self.crit(book_ratings_pred2.squeeze(1), book_rating)
        movie_ratings_pred1 = self.modelB(movie_user_embeddings, movie_item_embeddings)
        lossB1 = self.crit(movie_ratings_pred1.squeeze(1), movie_rating)
        movie_ratings_pred2 = self.modelA(movie_user_embeddings, movie_item_embeddings, dual=True)
        lossB2 = self.crit(movie_ratings_pred2.squeeze(1), movie_rating)
        lossA = (1-self.alpha)*lossA1 + self.alpha*Variable(lossA2.data, requires_grad=False)
        lossB = (1-self.alpha)*lossB1 + self.alpha*Variable(lossB2.data, requires_grad=False)
        lossA.backward(retain_graph=True)
        lossB.backward(retain_graph=True)
        orth_loss_A, orth_loss_B = torch.zeros(1), torch.zeros(1)
        reg = 1e-6
        for name, param in self.modelA.bridge.named_parameters():
            if 'bias' not in name:
                param_flat = param.view(param.shape[0], -1)
                sym = torch.mm(param_flat, torch.t(param_flat))
                sym -= torch.eye(param_flat.shape[0])
                orth_loss_A = orth_loss_A + (reg * sym.abs().sum())
        orth_loss_A.backward()
        for name, param in self.modelB.bridge.named_parameters():
            if 'bias' not in name:
                param_flat = param.view(param.shape[0], -1)
                sym = torch.mm(param_flat, torch.t(param_flat))
                sym -= torch.eye(param_flat.shape[0])
                orth_loss_B = orth_loss_B + (reg * sym.abs().sum())
        orth_loss_B.backward()
        self.optA.step()
        self.optB.step()
        if self.config['use_cuda'] is True:
            lossA = lossA.data.cpu().numpy()[0]
            lossB = lossB.data.cpu().numpy()[0]
            orth_loss_A = orth_loss_A.data.cpu().numpy()[0]
            orth_loss_B = orth_loss_B.data.cpu().numpy()[0]
        else:
            lossA = lossA.data.numpy()
            lossB = lossB.data.numpy()
            orth_loss_A = orth_loss_A.data.numpy()
            orth_loss_B = orth_loss_B.data.numpy()
        return lossA + lossB + orth_loss_A + orth_loss_B

    def train_an_epoch(self, train_book_loader, train_movie_loader, epoch_id):
        self.modelA.train()
        self.modelB.train()
        total_loss = 0
        for book_batch, movie_batch in zip(train_book_loader, train_movie_loader):
            assert isinstance(book_batch[0], torch.LongTensor)
            book_rating, book_user_embeddings, book_item_embeddings = Variable(book_batch[2]), Variable(book_batch[3]), Variable(book_batch[4])
            movie_rating, movie_user_embeddings, movie_item_embeddings = Variable(movie_batch[2]), Variable(movie_batch[3]), Variable(movie_batch[4])
            book_rating = book_rating.float()
            movie_rating = movie_rating.float()
            if self.config['use_cuda'] is True:
                book_rating = book_rating.cuda()
                movie_rating = movie_rating.cuda()
                book_user_embeddings = book_user_embeddings.cuda()
                book_item_embeddings = book_item_embeddings.cuda()
                movie_user_embeddings = movie_user_embeddings.cuda()
                movie_item_embeddings = movie_item_embeddings.cuda()
            loss = self.train_single_batch(book_user_embeddings, book_item_embeddings, book_rating,
                                           movie_user_embeddings, movie_item_embeddings, movie_rating)
            total_loss += loss

    def evaluate(self, evaluate_book_data, evaluate_movie_data, epoch_id):
        self.modelA.eval()
        self.modelB.eval()
        book_user, book_item, book_user_embeddings, book_item_embeddings, \
            book_golden = evaluate_book_data[0], evaluate_book_data[1], \
                Variable(evaluate_book_data[2]), Variable(evaluate_book_data[3]), evaluate_book_data[4]
        movie_user, movie_item, movie_user_embeddings, movie_item_embeddings, \
            movie_golden = evaluate_movie_data[0], evaluate_movie_data[1], \
                Variable(evaluate_movie_data[2]), Variable(evaluate_movie_data[3]), evaluate_movie_data[4]
        if self.config['use_cuda'] is True:
            book_user_embeddings = book_user_embeddings.cuda()
            book_item_embeddings = book_item_embeddings.cuda()
            movie_user_embeddings = movie_user_embeddings.cuda()
            movie_item_embeddings = movie_item_embeddings.cuda()
        book_scores = self.modelA(book_user_embeddings, book_item_embeddings)
        book_scores = book_scores.detach().numpy()
        movie_scores = self.modelB(movie_user_embeddings, movie_item_embeddings)
        movie_scores = movie_scores.detach().numpy()
        book_MSE = mean_squared_error(book_golden, book_scores)
        book_MAE = mean_absolute_error(book_golden, book_scores)
        movie_MSE = mean_squared_error(movie_golden, movie_scores)
        movie_MAE = mean_absolute_error(movie_golden, movie_scores)

       # unique_book_user = list(set(book_user))
       # unique_movie_user = list(set(movie_user))
       # book_recommend, movie_recommend = [], []
        #book_precision, movie_precision, book_recall, movie_recall = [], [], [], []
        #for index in range(len(book_user)):
         # book_recommend.append((book_user[index],book_item[index],book_golden[index],book_scores[index]))
       # for index in range(len(movie_user)):
          #  movie_recommend.append((movie_user[index],movie_item[index],movie_golden[index],movie_scores[index]))
        #for user in unique_book_user:
           # user_ratings = [x for x in book_recommend if x[0]==user]
           # user_ratings.sort(key=lambda x:x[3], reverse=True)
           # n_rel = sum((true_r >= 0.5) for (_, _, true_r, _) in user_ratings)
           # n_rec_k = sum((est >= 0.5) for (_, _, _, est) in user_ratings)
           # n_rel_and_rec_k = sum(((true_r >= 0.5) and (est >= 0.5))
                           # for (_, _, true_r, est) in user_ratings)
            # book_precision.append(n_rel_and_rec_k / n_rec_k if n_rec_k!=0 else 1)
            # book_recall.append(n_rel_and_rec_k / n_rel if n_rel!=0 else 1)
            # book_precision = np.mean(book_precision)
            # book_recall = np.mean(book_recall)
        #for user in unique_movie_user:
            #user_ratings = [x for x in movie_recommend if x[0]==user]
           # user_ratings.sort(key=lambda x:x[3], reverse=True)
           # n_rel = sum((true_r >= 0.5) for (_, _, true_r, _) in user_ratings)
            # n_rec_k = sum((est >= 0.5) for (_, _, _, est) in user_ratings)
            # n_rel_and_rec_k = sum(((true_r >= 0.5) and (est >= 0.5))
                             # for (_, _, true_r, est) in user_ratings)
           # movie_precision.append(n_rel_and_rec_k / n_rec_k if n_rec_k!=0 else 0)
          #  movie_recall.append(n_rel_and_rec_k / n_rel if n_rel!=0 else 1)
          #  movie_precision = np.mean(movie_precision)
        #movie_recall = np.mean(movie_recall)
       # book_ndcg = ndcg_score(book_golden, book_scores, k=10)
       # movie_ndcg = ndcg_score(movie_golden, movie_scores, k=10)
        print('[Book Evaluating Epoch {}] BMSE = {:.4f}, BMAE = {:.4f}'.format(epoch_id, book_MSE, book_MAE)) #, book_precision, book_recall))
        print('[Movie Evaluating Epoch {}] MMSE = {:.4f}, MMAE = {:.4f}'.format(epoch_id, movie_MSE, movie_MAE))#, movie_precision, movie_recall))
        Book_MSE_list.append(book_MSE)
        Book_MAE_list.append(book_MAE)
       # Book_Precision_list.append(book_precision)
      #  Book_Recall_list.append(book_recall)
        Music_MSE_list.append(movie_MSE)
        Music_MAE_list.append(movie_MAE)
        # Music_Precision_list.append(movie_precision)
        # Music_Recall_list.append(movie_recall)

        return book_MSE, book_MAE, movie_MSE, movie_MAE

    def save(self, dirname, filename):
        with open(os.path.join(dirname, filename)+'A', 'wb') as f:
            torch.save(self.modelA.state_dict(), f)
        with open(os.path.join(dirname, filename)+'B', 'wb') as f:
            torch.save(self.modelB.state_dict(), f)

Data

In [20]:
import torch
import random
from copy import deepcopy
from torch.utils.data import DataLoader, Dataset

In [21]:
random.seed(0)


class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    def __init__(self, user_tensor, item_tensor, target_tensor, user_embedding_tensor, item_embedding_tensor):
        """
        args:

            target_tensor: torch.Tensor, the corresponding rating for <user, item> pair
        """
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor
        self.user_embedding_tensor = user_embedding_tensor
        self.item_embedding_tensor = item_embedding_tensor

    def __getitem__(self, index):
        return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index], self.user_embedding_tensor[index], self.item_embedding_tensor[index]

    def __len__(self):
        return self.user_tensor.size(0)


class SampleGenerator(object):
    """Construct dataset for NCF"""

    def __init__(self, ratings):
        """
        args:
            ratings: pd.DataFrame, which contains 5 columns
            ['userId', 'itemId', 'rating', 'item_embedding', 'user_embedding']
        """
        assert 'userId' in ratings.columns
        assert 'itemId' in ratings.columns
        assert 'rating' in ratings.columns
        assert 'item_embedding' in ratings.columns
        assert 'user_embedding' in ratings.columns

        self.ratings = ratings
        self.normalize_ratings = self._normalize(ratings)
        self.user_pool = set(self.ratings['userId'].unique())
        self.item_pool = set(self.ratings['itemId'].unique())
        self.train_ratings, self.test_ratings = self._split_loo(self.normalize_ratings)
        #self.train_ratings, self.test_ratings = self._split_loo(self.ratings)

    def _normalize(self, ratings):
        """normalize into [0, 1] from [0, max_rating]"""
        ratings = deepcopy(ratings)
        max_rating = ratings.rating.max()
        ratings['rating'] = ratings.rating * 1.0 / max_rating
        return ratings

    def _split_loo(self, ratings):
        """leave one out train/test split """
        #ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)
        cut = 9 * len(ratings) // 10
        train = ratings[:cut]
        test = ratings[cut:]
        return train[['userId', 'itemId', 'rating', 'user_embedding', 'item_embedding']], test[['userId', 'itemId', 'rating', 'user_embedding','item_embedding']]

    def instance_a_train_loader(self, batch_size):
        """instance train loader for one training epoch"""
        users, items, ratings, user_embeddings, item_embeddings = [], [], [], [], []
        train_ratings = self.train_ratings
        train_ratings = train_ratings.sample(frac=1)
        for row in train_ratings.itertuples():
            users.append(int(row.userId))
            items.append(int(row.itemId))
            ratings.append(float(row.rating))
            user_embeddings.append(row.user_embedding)
            item_embeddings.append(row.item_embedding)
        dataset = UserItemRatingDataset(user_tensor=torch.LongTensor(users),
                                        item_tensor=torch.LongTensor(items),
                                        target_tensor=torch.FloatTensor(ratings),
                                        user_embedding_tensor=torch.FloatTensor(user_embeddings),
                                        item_embedding_tensor=torch.FloatTensor(item_embeddings))
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

    @property
    def evaluate_data(self):
        """create evaluate data"""
        test_ratings = self.test_ratings
        test_users, test_items, test_user_embeddings, test_item_embeddings, test_golden = [], [], [], [], []
        for row in test_ratings.itertuples():
            test_users.append(int(row.userId))
            test_items.append(int(row.itemId))
            test_golden.append(float(row.rating))
            test_user_embeddings.append(row.user_embedding)
            test_item_embeddings.append(row.item_embedding)
        return [test_users, test_items, torch.FloatTensor(test_user_embeddings), torch.FloatTensor(test_item_embeddings), test_golden]

In [22]:
import argparse
import pandas as pd

In [23]:
from numpy import nan

In [24]:
book = book.fillna(0)
movie = movie.fillna(0)

In [25]:
book.head(1)

,userId,itemId,rating,item_embedding,user_embedding
0,0,0,5.0,"[1.0, 0.4375, 0.4373, 0.0011]","[0.0041, 0.0, 0.1111, 0.0]"


In [26]:
movie.head(1)

,userId,itemId,rating,user_embedding,item_embedding
0,0,0,4.0,"[0.0041, 0.0, 0.1111, 0.0]","[0.819, 0.045, 0.8205, 0.5205]"


In [27]:
movie.dtypes

userId              int64
itemId              int64
rating            float64
user_embedding     object
item_embedding     object
dtype: object

In [306]:
#movie["rating"] = pd.to_numeric(movie["rating"], errors='coerce')
#movie["userId"] = pd.to_numeric(movie["userId"], errors='coerce')
#movie["itemId"] = pd.to_numeric(movie["itemId"], errors='coerce')

In [307]:
#movie['rating'] = movie['rating'].str.replace(r'\s+\(.+?\)', '', regex=True)

In [28]:
book.dtypes

userId              int64
itemId              int64
rating            float64
item_embedding     object
user_embedding     object
dtype: object

In [ ]:
parser = argparse.ArgumentParser('DITL')
# Path Arguments
parser.add_argument('-f')
parser.add_argument('--num_epoch', type=int, default=200,help='number of epoches')
parser.add_argument('--batch_size', type=int, default=1024,help='batch size')
parser.add_argument('--lr', type=int, default=1e-2,help='learning rate')
parser.add_argument('--latent_dim', type=int, default=3,help='latent dimensions')
parser.add_argument('--alpha', type=int, default=0.003,help='dual learning rate')
parser.add_argument('--cuda', action='store_true',help='use of cuda')
args = parser.parse_args()

def dictionary(terms):
    term2idx = {}
    idx2term = {}
    for i in range(len(terms)):
        term2idx[terms[i]] = i
        idx2term[i] = terms[i]
    return term2idx, idx2term

mlp_config = {'num_epoch': args.num_epoch,
              'batch_size': args.batch_size,
              'optimizer': 'sgd',
              'lr': args.lr,
              'latent_dim': args.latent_dim,
              'nlayers':1,
              'alpha':args.alpha,
              'layers': [2*args.latent_dim,64,args.latent_dim],  # layers[0] is the concat of latent user vector & latent item vector
              'use_cuda': args.cuda,
              'pretrain': False,}

#Load Datasets
#book = pd.read_csv('/content/drive/MyDrive/objective 3/Dataset/New_created/df_music_final_90.csv')
#movie = pd.read_csv('/content/drive/MyDrive/objective 3/Dataset/New_created/df_movie_final_90.csv')
#movie = movie.fillna(0)
#book = book.fillna(0)
book['user_embedding']= book['user_embedding'].map(str)
book['item_embedding']= book['item_embedding'].map(str)
movie['user_embedding']= movie['user_embedding'].map(str)
movie['item_embedding']= movie['item_embedding'].map(str)
#book['user_embedding'] = book['user_embedding'].apply(lambda x: ''.join(map(str, x)))
#book['item_embedding'] = book['item_embedding'].apply(lambda x: ''.join(map(str, x)))
#movie['user_embedding'] = movie['user_embedding'].apply(lambda x: ''.join(map(str, x)))
#movie['item_embedding'] = movie['item_embedding'].apply(lambda x: ''.join(map(str, x)))
book['user_embedding'] = book['user_embedding'].map(eval)
book['item_embedding'] = book['item_embedding'].map(eval)
movie['user_embedding'] = movie['user_embedding'].map(eval)
movie['item_embedding'] = movie['item_embedding'].map(eval)
#book['rating']= book['rating'].map(str)
sample_book_generator = SampleGenerator(ratings=book)
evaluate_book_data = sample_book_generator.evaluate_data
sample_movie_generator = SampleGenerator(ratings=movie)
evaluate_movie_data = sample_movie_generator.evaluate_data
config = mlp_config
engine = Engine(config)
best_MSE = 1
for epoch in range(config['num_epoch']):
    print('Epoch {} starts !'.format(epoch))
    print('-' * 80)
    train_book_loader = sample_book_generator.instance_a_train_loader(config['batch_size'])
    train_movie_loader = sample_movie_generator.instance_a_train_loader(config['batch_size'])
    engine.train_an_epoch(train_book_loader, train_movie_loader, epoch_id=epoch)
    book_MSE, book_MAE, movie_MSE, movie_MAE = engine.evaluate(evaluate_book_data, evaluate_movie_data, epoch_id=epoch)

MLP(
  (fc_layers): ModuleList(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
  (affine_output): Linear(in_features=3, out_features=1, bias=True)
  (logistic): Sigmoid()
  (bridge): Linear(in_features=3, out_features=3, bias=True)
)
Epoch 0 starts !
--------------------------------------------------------------------------------


In [ ]:
RMSE_book= np.sqrt((sum(Book_MSE_list)/len(Book_MSE_list)))
RMSE_book

In [ ]:
RMSE_music= np.sqrt((sum(Music_MSE_list)/len(Music_MSE_list)))
RMSE_music

In [ ]:
MAE_book= sum(Book_MAE_list)/len(Book_MAE_list)
MAE_book

In [ ]:
MAE_music= sum(Music_MAE_list)/len(Music_MAE_list)
MAE_music

In [ ]:
#df.head(10)

In [ ]:
#df["Recall"] = df["Recall"].astype(str).astype(int)

In [ ]:

# MLP for Pima Indians Dataset Serialize to JSON and HDF5
#from tensorflow.keras.models import model_from_json

In [ ]:
#model_json = engine.to_json()
#with open("engine.json", "w") as json_file:
    #json_file.write(model_json)

In [ ]:
#hist_df = engine.save('/content/drive/MyDrive/objective 3/Dataset/created dataset/')

In [ ]:
#hist_df = pd.DataFrame(engine.history)